In [1]:
import pandas as pd

In [2]:
df1 = pd.read_excel("../Data/DDW_PCA0000_2011_Indiastatedist.xlsx", sheet_name='Sheet1', usecols = ['State', 'Level', 'Name', 'TRU', 'TOT_P', 'TOT_M', 'TOT_F'])
df1 = df1[df1["TRU"] == "Total"]
#Replacing State codes with state names in census data
for idx, item in df1[df1["Level"] == "STATE"].iterrows():
    df1.replace(item["State"], item["Name"], inplace = True)
df1 = df1[df1["Level"] == "STATE"]
df1.reset_index(inplace = True)
#Updating State names in census data according to vaccine data
df1.replace({'&': 'AND'}, regex=True, inplace = True)
df1.replace({'NCT OF DELHI': 'DELHI'}, regex=True, inplace = True)
df1.replace({'DAMAN AND DIU': 'Dadra and Nagar Haveli and Daman and Diu'.upper()}, regex=True, inplace = True)
df1.replace('DADRA AND NAGAR HAVELI', 'Dadra and Nagar Haveli and Daman and Diu'.upper(), inplace = True)
df1 = df1.groupby(['State', 'Level', 'Name'], as_index=False).agg('sum')

In [3]:
df2 = pd.read_csv("../Data/cowin_vaccine_data_districtwise.csv", skiprows = [1])
df2 = df2.fillna(0)
df2 = df2.groupby(['State_Code', 'State', 'District_Key', 'District'], as_index=False).agg('sum')
df2 = df2.apply(lambda x: x.astype(str).str.upper())     #to make every state in uppercase

In [4]:
state = {"stateid" : [], "vaccinationratio" : [], "populationratio" : [], "ratioofratios" : []}
skipped = 0
for idx, row in df1.iterrows():
    try:
        scode = df2.State_Code[df2["State"] == row["Name"]].astype('category').values[0]
        state["stateid"].append(scode)
    except IndexError:
        skipped += 1
        continue
        
    #vaccination ratio calculation
    f = list(df2[df2["State"] == row["Name"]]["14-08-2021.6"].astype('category').values)
    f = [int(i) for i in f]
    m = list(df2[df2["State"] == row["Name"]]["14-08-2021.5"].astype('category').values)
    m = [int(i) for i in m]
    state["vaccinationratio"].append(sum(f)/sum(m))

    #population ratio calculation
    pratio = row["TOT_F"] / row["TOT_M"]
    state["populationratio"].append(pratio)
    
    #ratio of ratios
    rr = state["vaccinationratio"][idx - skipped] / state["populationratio"][idx - skipped]
    state["ratioofratios"].append(rr)

In [5]:
df = pd.DataFrame(state)

In [6]:
df.to_csv("../Final Outputs/vaccination-population-ratio-state.csv", index = False)